In [2]:
import json
import random
import re
import requests
import pandas as pd


In [3]:
# url and user to use the API rest

endponint_get = 'https://7b8uflffq0.execute-api.us-east-1.amazonaws.com/game/get_params'

endponint_post = 'https://7b8uflffq0.execute-api.us-east-1.amazonaws.com/game/check_results'

user = ('juan.bedoya','6b2a4f8d28204cf0a3286d1f9213ecfc')

In [6]:
# open data base (clean words)

with open('Data_Base_Final_Words.txt', 'r',encoding= 'utf-8') as data_file:

  logs_text = data_file.read()

words_final_list = logs_text.split()
print(len(words_final_list))

32430


In [103]:
def choose_word(words_list):
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                'n','ñ', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    position_letters_dict = posittion_dict(alphabet,words_list)

    letter_list = [i for i in position_letters_dict]
    df_words_frecuency= pd.DataFrame(list(zip(letter_list)),
                        columns =['letters'])

    df_words_frecuency.set_index('letters')

    for i in range(len(words_list[0])):

        df_words_frecuency['position_{}'.format(i)] = [position_letters_dict[j].count(i)/len(filt) for j in position_letters_dict]

    df_words_frecuency= df_words_frecuency.set_index('letters')
    chosee_weight = 0
    for word in words_list:
        weight = 0
        for position,letter in enumerate(word):
            weight += df_words_frecuency.loc[letter,'position_{}'.format(position)] / word[position:].count(letter)
        if weight > chosee_weight:
            select_word = word
            chosee_weight = weight
    
    return select_word
    
def posittion_dict(letters_list,words_list):
    dicts = {}
    for letter in letters_list:
        for word in words_list:
            for letter1 in range(len(word)):
                if letter == word[letter1]:
                    if letter not in dicts:
                        dicts[letter] = []
                    dicts[letter].append(letter1)
                    
    return dicts

def words_specific_length(words_list,long):    
    words = []
    for word in words_list:
        if len(word) == long:
            words.append(word)
    return words

def words_by_vowels(words_list,num_vowels):
    word_list_by_vowels = []
    for word in words_list:
        count_vowels = len(re.findall("[aeiou]", word))
        if count_vowels == num_vowels:
            word_list_by_vowels.append(word)
    return word_list_by_vowels

def dic_wrong_positions(positions_list,wrong_positions_letter,last_word):
    global dic_wrong_positions_letters
    for letters_in_word in wrong_positions_letter:
        for letter_position in range(len(last_word)):
            if letters_in_word == last_word[letter_position] and positions_list[letter_position] == False:
                if letters_in_word not in dic_wrong_positions_letters:
                    dic_wrong_positions_letters[letters_in_word] = []
                dic_wrong_positions_letters[letters_in_word].append(letter_position) 
    return dic_wrong_positions_letters
    
def wrong_letters(positions_list,dict_letters_wposition,last_word):
    global list_wrong_letters
    for position, letter in enumerate(last_word):
        if letter not in dict_letters_wposition and positions_list[position] != True:
            list_wrong_letters.append(letter)
    return list_wrong_letters

def correct_position_words(position_list,last_word,words_list):
    possible_words = []
    for word in words_list:
        count = 0
        for position, letter in enumerate(word):
            if letter == last_word[position] and position_list[position] == True:
                count += 1
        if count == sum(position_list):
            possible_words.append(word)
    return possible_words

def words_without_wrong_letters(words_list,letters_list):
    possible_words = words_list[:]
    for word in words_list:
        for letter in letters_list:
            if letter in word:
                possible_words.remove(word)
                break
    return possible_words

def words_without_letters_wposition(words_list,dict_letters_wposition):
    possible_words = words_list[:]
    for word in words_list:
        for position ,letter in enumerate(word):
            if letter in dict_letters_wposition:
                if position in dict_letters_wposition[letter]:
                    possible_words.remove(word)
                    break
    return(possible_words)

def words_with_letters_in(words_list,dict_letters_wpossition):
    list_letters_in = list(dict_letters_wpossition)
    possible_words = words_list[:]
    for word in words_list:
        for letter in list_letters_in:
            find = re.findall(letter,word)
            if find == []:
                possible_words.remove(word)
                break
    return possible_words
        
def find_next_word(words_list,last_word,position_list,wrong_positions_letter):

    create_dic_wrong_positions = dic_wrong_positions(position_list,wrong_positions_letter,last_word)

    list_wrong_letters = wrong_letters(position_list,create_dic_wrong_positions,last_word)

    words_with_correct_letters = correct_position_words(position_list,last_word,words_list)

    words_without_wletters = words_without_wrong_letters(words_with_correct_letters,list_wrong_letters)

    words_without_wposition = words_without_letters_wposition(words_without_wletters,create_dic_wrong_positions)

    words_with_wpossition_letters = words_with_letters_in(words_without_wposition,create_dic_wrong_positions)

    return words_with_wpossition_letters





In [4]:
#start the game

response_get_letter = requests.get(endponint_get, auth = user)
print(response_get_letter.json())


{'id': '62b61e70ce8881b753923b44', 'length_word': 6, 'vowels': 2, 'consonants': 4}


In [103]:
print(response_get_letter.json())
json = response_get_letter.json()
print(json['vowels'])

{'id': '62b61e70ce8881b753923b44', 'length_word': 6, 'vowels': 2, 'consonants': 4}
2


In [104]:
# local test choose random word 

dic_wrong_positions_letters = {}
dic_correct_positions_letters = {}
list_wrong_letters = []

print(json["vowels"])

dummy = words_specific_length(words_final_list,json["length_word"])
#print(dummy)
filt = []
filt = words_by_vowels(dummy,json['vowels'])
#print(filt)

random_word = random.choice(filt)
random_word1 = random.choice(filt)

cont = 0
print(random_word)
print(random_word1)


2
ballet
cintas


In [82]:
json_response ={
    "result_word" : "arseon"
}

In [93]:
response_post_word = requests.post(endponint_post,json = json_response,auth = user)
print(response_post_word.json())

{'word_sent': 'sirven', 'score': 1.0, 'try_datetime': '2022-06-24T21:53:16.236826', 'position_array': [True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 3}


In [94]:
print(response_post_word.json())
dic_wrong_positions_letters = {}
dic_correct_positions_letters = {}
list_wrong_letters = []

{'word_sent': 'sirven', 'score': 1.0, 'try_datetime': '2022-06-24T21:53:16.236826', 'position_array': [True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 3}


In [95]:
json = response_post_word.json()

find = find_next_word(filt,json["word_sent"],json["position_array"],json["right_letters_in_wrong_positions"])
print(find)
random_word1 = random.choice(find)


['sirven']


In [92]:
json_response ={
    "result_word" : random_word1
}
print(json_response)

{'result_word': 'sirven'}


In [207]:
# local test choose random word 

dic_wrong_positions_letters = {}
dic_correct_positions_letters = {}
list_wrong_letters = []

json = {
  "id": "string",
  "length_word": 4,
  "vowels": 1,
  "consonants": 3
}

words_list_for_game = words_specific_length(words_final_list,json["length_word"])
#print(dummy)
words_list_for_game = words_by_vowels(words_list_for_game,json["vowels"])
#print(filt)

random_word = random.choice(words_list_for_game)

#random_word1 = random.choice(filt)
random_word1 = choose_word(words_list_for_game)
cont = 1
print(random_word)
print(random_word1)

while random_word1 != random_word:
  array = []
  for pos,let in enumerate(random_word1):
    if let == random_word[pos]:
      array.append(True)
    else:
      array.append(False)
  letters = []
  for position, letter in enumerate(random_word1):
    if letter in random_word and array[position] == False:
      letters.append(letter)
  # print(array)
  # print(letters)
  json = {
    "word_sent": random_word1,
    "score": 0.233,
    "try_datetime": '21-06-2022',
    "position_array": array,
    "right_letters_in_wrong_positions": letters
  }

  cont += 1
  print(cont)

  find = find_next_word(words_list_for_game,json["word_sent"],json["position_array"],json["right_letters_in_wrong_positions"])
  #print(find)
  random_word1 = choose_word(find)
  print(random_word)
  print(random_word1)



kirk
sant
2
kirk
folk
3
kirk
peck
4
kirk
kirk


In [102]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
            'n','ñ', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

json = {
  "id": "string",
  "length_word": 6,
  "vowels": 2,
  "consonants": 3
}

dummy = words_specific_length(words_final_list,json["length_word"])
#print(dummy)
filt = words_by_vowels(dummy,json["vowels"])

position_letters_dict = posittion_dict(alphabet,filt)
print(position_letters_dict['a'].count(0))

letter_list = [i for i in position_letters_dict]

df_words_frecuency= pd.DataFrame(list(zip(letter_list)),
                     columns =['letters'])

#print(df_words_frecuency)
df_words_frecuency.set_index('letters')

for i in range(json["length_word"]):

    df_words_frecuency['position_{}'.format(i)] = [position_letters_dict[j].count(i)/len(filt) for j in position_letters_dict]



df_words_frecuency= df_words_frecuency.set_index('letters')
#print(df_words_frecuency)
letter= 'a'
posi = 0
print(df_words_frecuency.loc[letter,'position_{}'.format(posi)])
    

28
0.02199528672427337


In [100]:
a = max(df_words_frecuency['position_0'].values)
b = df_words_frecuency.loc[df_words_frecuency['position_0']==a].index.values

print(b)

['c']
